<a href="https://colab.research.google.com/github/beethogedeon/rag-afcon2024-mixtral-8x7b/blob/master/RAG_WIth_Mistral_8x7b_AFCON_2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install necessary librairies

In [2]:
!pip install -U chromadb langchain sentence_transformers bitsandbytes tiktoken openai accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.0/509.0 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.0/798.0 kB 48.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 11.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 77.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.9/224.9 kB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 36.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 94.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.3/60.3 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.6/40.6 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 100.3 MB/

In [3]:
!pip install -U transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 31.7 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2


Import Librairies

In [4]:
import torch
from torch import bfloat16
import transformers
from transformers import AutoTokenizer, pipeline

from langchain.llms import HuggingFacePipeline
from langchain.document_loaders import RecursiveUrlLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma

from bs4 import BeautifulSoup as Soup

from google.colab import userdata

In [5]:
if torch.cuda.is_available():
  gpu_name = torch.cuda.get_device_name(torch.cuda.current_device())
  total_memory = torch.cuda.get_device_properties(0).total_memory
  total_memory_gb = total_memory / (1024**3) # Converting memory to Gb
  print("GPU is available. \nUsing GPU")
  print("\nGPU Name:", gpu_name)
  print(f"Total GPU Memory: {total_memory_gb:.2f} GB")
  device = torch.device('cuda')
else:
  print("GPU is not available. \nUsing CPU")
  device = torch.device('cpu')

GPU is available. 
Using GPU

GPU Name: Tesla T4
Total GPU Memory: 14.75 GB


In [6]:
url_to_scrap = "https://www.cafonline.com/caf-africa-cup-of-nations/"

In [7]:
loader = RecursiveUrlLoader(
    url=url_to_scrap, max_depth=3, extractor=lambda x: Soup(x, "html.parser").text
)

In [8]:
docs = loader.load()

In [9]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=128,
    chunk_overlap=20,
    length_function=len,
    is_separator_regex=False,
)

In [10]:
texts = text_splitter.split_documents(docs)

In [11]:
embedding_model = OpenAIEmbeddings(
    openai_api_key=userdata.get('OPENAI_API_KEY')
)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. Use langchain_openai.OpenAIEmbeddings instead.
  warn_deprecated(


In [12]:
# Creating and indexed database
chroma_database = Chroma.from_documents(texts,
embedding_model,
persist_directory = 'db')

In [13]:
retriever = chroma_database.as_retriever()

In [14]:
quantization_config = transformers.BitsAndBytesConfig(load_in_4bit = True,
bnb_4bit_quant_type = 'nf4',
bnb_4bit_use_double_quant = True,
bnb_4bit_compute_dtype = bfloat16)

In [15]:
llm = HuggingFacePipeline.from_model_id(
    model_id='mistralai/Mixtral-8x7B-v0.1',
    task = 'text-generation',
    device=0,
    model_kwargs={'temperature': .3,
                  'max_length': 1024,
                  'quantization_config': quantization_config,
                  },
    device_map = "auto"
    )

tokenizer_config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/720 [00:00<?, ?B/s]

...

model-00010-of-00019.safetensors:   0%|          | 0.00/4.90G [00:00<?, ?B/s]

model-00011-of-00019.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:983: UserWarning: Not enough free disk space to download the file. The expected file size is: 4983.00 MB. The target location /root/.cache/huggingface/hub only has 526.14 MB free disk space.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:983: UserWarning: Not enough free disk space to download the file. The expected file size is: 4983.00 MB. The target location /root/.cache/huggingface/hub/models--mistralai--Mixtral-8x7B-v0.1/blobs only has 526.14 MB free disk space.
  warnings.warn(


model-00012-of-00019.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

OSError: [Errno 28] No space left on device